In [5]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout

df = pd.read_csv('merged_data_datetime.csv')
df['开盘时间'] = pd.to_datetime(df['开盘时间'])

# Data scaling
scaler = MinMaxScaler(feature_range=(0, 1))
df_scaled = scaler.fit_transform(df['收盘价'].values.reshape(-1,1))
look_back = 100
def create_dataset(data, look_back):
    X, Y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        Y.append(data[i + look_back, 0])
    return np.array(X), np.array(Y)

X, Y = create_dataset(df_scaled, look_back)

# partitioned data set
train_size = int(len(X) * 0.8)
test_size = len(X) - train_size
trainX, testX = X[:train_size], X[train_size:]
trainY, testY = Y[:train_size], Y[train_size:]

# Reshaping input data to adapt to LSTM networks
trainX = trainX.reshape((trainX.shape[0], trainX.shape[1], 1))
testX = testX.reshape((testX.shape[0], testX.shape[1], 1))

In [6]:
# LSTM Model Definition
model = Sequential()
model.add(LSTM(60, return_sequences=True, input_shape=(look_back, 1)))
model.add(Dropout(0.2))
model.add(LSTM(80, return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(120, return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer='adam', loss='mean_squared_error')


In [ ]:
model.fit(trainX, trainY, epochs=10, batch_size=32)

Epoch 1/10
1359/1359 [==============================] - 116s 84ms/step - loss: 0.0014
Epoch 2/10
1359/1359 [==============================] - 116s 85ms/step - loss: 2.3857e-04
Epoch 3/10
1359/1359 [==============================] - 117s 86ms/step - loss: 2.0875e-04
Epoch 4/10
 940/1359 [===================>..........] - ETA: 35s - loss: 1.9370e-04

In [ ]:
from sklearn.metrics import mean_squared_error

# Predicting with the LSTM model in the test dataset
test_predicted = model.predict(testX)

# Rescaling the predicted values back to the original prices
test_predicted_actual = scaler.inverse_transform(test_predicted)
testY_actual = scaler.inverse_transform(testY.reshape(-1, 1))

# Calculating the Mean Squared Error
mse = mean_squared_error(testY_actual, test_predicted_actual)
print(f'Mean Squared Error: {mse}')

# Calculating the MAPE
mape = np.mean(np.abs(test_predicted_actual - testY_actual)/np.abs(testY_actual))
print(f'Mean Absolute Percentage Error: {mape}')

In [ ]:
import matplotlib.pyplot as plt
# Plotting the comparison between actual and predicted prices
plt.figure(figsize=(10,6))
plt.plot(testY_actual, label='Actual Prices')
plt.plot(test_predicted_actual, label='Predicted Prices', alpha=0.7)
plt.title('Comparison of Actual and Predicted Prices')
plt.xlabel('Time')
plt.ylabel('Price')
plt.legend()
plt.show()